IDENYA ADALAH FILE KELUAR MASUK
1. proses dulu si cosine doang outputnya list down mana x_test yg udah ada di y_train (score apa nya tinggi gitu) dan buat file baru dengan label yes/no
2. proses ini ngambil data dari proses no 1 khusus yang no aja jadi yg udah bener ga usah diproses lagi...
3. terus begitu sampe terakhir model ML jalan

In [1]:
import pandas as pd
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import accuracy_score
from rapidfuzz import process, fuzz

# Load data
train_df = pd.read_csv("data/Hospital_Train.csv")
test_df = pd.read_csv("data/Hospital_Test.csv")

# Clean data
train_df_clean = train_df.dropna(subset=["Hospital Name rev 2"])
X_train = train_df_clean["Hospital_Name (clean)"]
y_train = train_df_clean["Hospital Name rev 2"]
X_test = test_df["Hospital_Name (clean)"]
y_test = test_df["Hospital Name rev 2"]

# TF-IDF vectorization
vectorizer = TfidfVectorizer(ngram_range=(1, 2), sublinear_tf=True)
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

# Cosine similarity prediction
cos_sim_matrix = cosine_similarity(X_test_tfidf, X_train_tfidf)
cosine_top_indices = cos_sim_matrix.argmax(axis=1)
cosine_preds = y_train.iloc[cosine_top_indices].values
cosine_top_scores = cos_sim_matrix.max(axis=1)

# Format clean name
def format_clean_name(name):
    return re.sub(r'[^\w\s&]', '', name).upper().strip()

clean_names = X_test.apply(format_clean_name)

# FINAL PREDICTION LOGIC
final_preds = []
fuzzy_preds = []
fuzzy_scores = []
tfidf_zero_flags = []
tfidf_term_details = []

feature_names = vectorizer.get_feature_names_out()

for i in range(len(X_test)):
    cleaned_input = clean_names.iloc[i]
    actual = y_test.iloc[i]
    tfidf_row = X_test_tfidf[i]

    # Token analyzer
    tokens = vectorizer.build_analyzer()(cleaned_input)

    # TF-IDF vector content
    indices = tfidf_row.nonzero()[1]
    values = tfidf_row.data
    tfidf_tokens = {feature_names[idx] for idx in indices}
    tfidf_score_map = {feature_names[idx]: round(val, 4) for idx, val in zip(indices, values)}
    tfidf_term_details.append(tfidf_score_map)

    # Cek apakah ada token hilang
    missing_tokens = set(tokens) - tfidf_tokens
    has_zero = len(missing_tokens) > 0
    tfidf_zero_flags.append(has_zero)

    # Fuzzy matching
    fuzzy_match, fuzzy_score, _ = process.extractOne(
        cleaned_input,
        y_train.tolist(),  # fix: convert to list
        scorer=fuzz.token_sort_ratio
    )
    fuzzy_preds.append(fuzzy_match)
    fuzzy_scores.append(fuzzy_score)

    # FINAL DECISION FLOW
    if cosine_preds[i] == actual:
        final_preds.append(cosine_preds[i])
    elif has_zero:
        # Use fuzzy result as new input to cosine
        new_input_vector = vectorizer.transform([fuzzy_match])
        new_cos_sim = cosine_similarity(new_input_vector, X_train_tfidf)
        new_top_idx = new_cos_sim.argmax()
        new_cosine_pred = y_train.iloc[new_top_idx]

        if new_cosine_pred == actual:
            final_preds.append(new_cosine_pred)
            # print(f"[{i}] 🌀 Fuzzy → Cosine match: {fuzzy_match} → {new_cosine_pred}")
        else:
            final_preds.append(cleaned_input)
    elif fuzzy_match == actual:
        final_preds.append(fuzzy_match)
    else:
        final_preds.append(cleaned_input)

# Buat hasil dataframe
result_df = pd.DataFrame({
    "Hospital_Name_clean": X_test,
    "Actual": y_test,
    "Cosine_Pred": cosine_preds,
    "Fuzzy_Pred": fuzzy_preds,
    "Fuzzy_Score": fuzzy_scores,
    "Clean_Pred": clean_names,
    "Final_Pred": final_preds,
    "Cosine_Score": cosine_top_scores,
    "TF-IDF_Zero_Found": tfidf_zero_flags,
    "TF-IDF_Terms_Score": tfidf_term_details
})

# Bersihkan spasi ganda
for col in ["Final_Pred", "Fuzzy_Pred"]:
    result_df[col] = result_df[col].str.replace(r"\s+", " ", regex=True).str.strip()

# Tampilkan mismatch & evaluasi
mismatch_df = result_df[result_df["Actual"] != result_df["Final_Pred"]]
accuracy = accuracy_score(y_test, result_df["Final_Pred"])

print("\n🎯 Accuracy:", accuracy)
print("🔥 Total mismatch:", len(mismatch_df))
print("🧠 Total TF-IDF OOV:", sum(tfidf_zero_flags))
mismatch_df.head(10)



🎯 Accuracy: 0.8786407766990292
🔥 Total mismatch: 25
🧠 Total TF-IDF OOV: 148


,Hospital_Name_clean,Actual,Cosine_Pred,Fuzzy_Pred,Fuzzy_Score,Clean_Pred,Final_Pred,Cosine_Score,TF-IDF_Zero_Found,TF-IDF_Terms_Score
16,RS KHUSUS ORTHOEDI KARIMA UTAMA,RS KHUSUS ORTHOPEDI KARIMA UTAMA,RS KHUSUS BEDAH KARIMA UTAMA,RS KHUSUS ORTHOPEDI KARIMA UTAMA,98.412698,RS KHUSUS ORTHOEDI KARIMA UTAMA,RS KHUSUS ORTHOEDI KARIMA UTAMA,0.771471,True,"{'karima': 0.543, 'karima utama': 0.543, 'khus..."
34,RS THT PROKLAMASI,RS KHUSUS THT BEDAH PROKLAMASI,RS THT BEDAH PROKLAMASI,RS PROKLAMASI,86.666667,RS THT PROKLAMASI,RS THT PROKLAMASI,0.614295,False,"{'proklamasi': 0.4631, 'rs': 0.0869, 'rs tht':..."
40,HERMINA HOSPITALS MADIUN,RS HERMINA MADIUN,RS SANTA CLARA MADIUN,HARI NATIONAL HOSPITAL,69.565217,HERMINA HOSPITALS MADIUN,HERMINA HOSPITALS MADIUN,0.398326,True,"{'hermina': 0.4645, 'hospitals': 0.4444, 'madi..."
66,RSIA BUN,RS BUNDA JAKARTA,RS BUN,RS BUN,85.714286,RSIA BUN,RSIA BUN,0.587421,True,"{'bun': 0.8989, 'rsia': 0.4382}"
67,REVIEW DOR 22 -DOKUMEN LAPORAN OP SUDAH DIKIRI...,RS BETHESDA YOGYAKARTA,RS BETHESDA,RSUP PROF DR R D KANDOU MANADO,48.000000,REVIEW DOR 22 DOKUMEN LAPORAN OP SUDAH DIKIRIM...,REVIEW DOR 22 DOKUMEN LAPORAN OP SUDAH DIKIRIM...,0.388503,True,"{'22': 0.3309, 'bethesda': 0.2697, 'by': 0.344..."
86,HOME CLINIC,PRODIA,HOME CARE,MOM CLINIC,85.714286,HOME CLINIC,HOME CLINIC,0.562364,True,"{'clinic': 0.4424, 'home': 0.8968}"
88,RSSTELISABETH SEMARANG,RS ST ELISABETH SEMARANG,SILOAM HOSPITAL SEMARANG,RS ELISABETH SEMARANG,83.720930,RSSTELISABETH SEMARANG,RSSTELISABETH SEMARANG,0.533078,True,{'semarang': 1.0}
108,"THE ENT,VOICE DAN SNORING CLINIC",MOUNT ELIZABETH MEDICAL CENTRE SINGAPORE,THE ENT SPECIALIST CENTRE PARKWAY SINGAPORE,HE ENDOCRINE CLINIC,68.000000,THE ENTVOICE DAN SNORING CLINIC,THE ENTVOICE DAN SNORING CLINIC,0.890817,True,"{'clinic': 0.1893, 'dan': 0.2692, 'ent': 0.294..."
110,THE SPORE GASTROENTEROLOGY & LIVER INTERNIST C...,SPORE GASTROENTEROLOGY & LIVER INTERNIST CENTR...,SINGAPORE GASTROENTEROLOGY & LIVER INTERNIST C...,SINGAPORE GASTROENTEROLOGY & LIVER INTERNIST C...,78.571429,THE SPORE GASTROENTEROLOGY & LIVER INTERNIST C...,THE SPORE GASTROENTEROLOGY & LIVER INTERNIST C...,0.868745,True,"{'care': 0.2188, 'dan': 0.2315, 'gastroenterol..."
111,SPORTSLN ORHOPEDIC CLINIC GELEAGLES MEDICAL CA...,GLENEAGLES HOSPITAL,E-CLINIC,SOG ANDY TAN CLINIC FOR WOMEN GLENEAGLES MEDIC...,64.761905,SPORTSLN ORHOPEDIC CLINIC GELEAGLES MEDICAL CA...,SPORTSLN ORHOPEDIC CLINIC GELEAGLES MEDICAL CA...,0.364512,True,"{'cantre': 0.8493, 'clinic': 0.3645, 'medical'..."


In [ ]:
# Optional: save to Excel
result_df.to_excel("NLP_Cosine_Fuzzy_MLP_Pred.xlsx", index=False)
print("✅ Predictions saved to 'NLP_Cosine_Fuzzy_MLP_Pred.xlsx'")

✅ Predictions saved to 'NLP_Cosine_Fuzzy_Pred.xlsx'


In [7]:
from sklearn.neural_network import MLPClassifier

# Step 1: Siapkan data train (yang sudah bener)
correct_df = result_df[result_df["Actual"] == result_df["Final_Pred"]]
X_train_mlp = correct_df["Hospital_Name_clean"]
y_train_mlp = correct_df["Actual"]

# Step 2: Siapkan data test dari yang mismatch
X_test_mlp = mismatch_df["Hospital_Name_clean"]
y_test_mlp = mismatch_df["Actual"]

# Step 3: Vectorize
vectorizer_mlp = TfidfVectorizer(ngram_range=(1, 2), sublinear_tf=True)
X_train_vec = vectorizer_mlp.fit_transform(X_train_mlp)
X_test_vec = vectorizer_mlp.transform(X_test_mlp)

# Step 4: Train MLPClassifier
mlp_model = MLPClassifier(hidden_layer_sizes=(128,), max_iter=500, random_state=42)
mlp_model.fit(X_train_vec, y_train_mlp)

# Step 5: Predict mismatch
mlp_preds = mlp_model.predict(X_test_vec)

# Step 6: Update Final_Pred di result_df
for i, idx in enumerate(mismatch_df.index):
    result_df.loc[idx, "Final_Pred"] = mlp_preds[i]

# Step 7: Bersihkan ulang
result_df["Final_Pred"] = result_df["Final_Pred"].str.replace(r"\s+", " ", regex=True).str.strip()

# Step 8: Evaluasi ulang
mismatch_df_final = result_df[result_df["Actual"] != result_df["Final_Pred"]]
accuracy_final = accuracy_score(result_df["Actual"], result_df["Final_Pred"])

# Output
print("\n✅ AFTER MLP CLASSIFIER")
print("🎯 Accuracy:", accuracy_final)
print("🔥 Remaining mismatch:", len(mismatch_df_final))
print("🧠 Total TF-IDF OOV:", sum(tfidf_zero_flags))



✅ AFTER MLP CLASSIFIER
🎯 Accuracy: 0.883495145631068
🔥 Remaining mismatch: 24
🧠 Total TF-IDF OOV: 148


In [4]:
# Ambil vocab dari TF-IDF
feature_names = vectorizer.get_feature_names_out()

# Hitung skor TF-IDF untuk setiap baris di X_test
tfidf_term_details = []

for i in range(len(X_test)):
    tfidf_vector = X_test_tfidf[i]
    indices = tfidf_vector.nonzero()[1]
    values = tfidf_vector.data
    terms_scores = {feature_names[idx]: round(val, 4) for idx, val in zip(indices, values)}
    tfidf_term_details.append(terms_scores)


In [5]:
result_df["TF-IDF_Terms_Score"] = tfidf_term_details


In [6]:
# Tampilkan tf-idf token dan skornya untuk 1 baris
print("\nContoh skor TF-IDF token dari baris pertama:")
print(result_df.loc[3, "TF-IDF_Terms_Score"])


Contoh skor TF-IDF token dari baris pertama:
{'clinic': np.float64(0.3528), 'clinic orchard': np.float64(0.8221), 'orchard': np.float64(0.4468)}
